In [33]:
import pandas as pd
import numpy as np

from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier

import tensorflow as tf
from tensorflow import keras
from keras import regularizers
from keras import layers
from keras import optimizers
import keras.backend as K

from random import shuffle

In [34]:
def f1_metric(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [35]:
df = pd.read_csv(open("heart.csv"))
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0


In [36]:
# preprocessing

# converting all continuos data to categorical - using percentiles
columns = ("age", "trestbps", "chol", "thalach", "oldpeak")

for col in columns:
    df[col], tiles = pd.qcut(df[col], 4, labels = False, retbins = True, duplicates = "drop")
    print(col, " categories with bin edges: ", tiles)

age  categories with bin edges:  [29.  47.5 55.  61.  77. ]
trestbps  categories with bin edges:  [ 94. 120. 130. 140. 200.]
chol  categories with bin edges:  [126.  211.  240.  274.5 564. ]
thalach  categories with bin edges:  [ 71.  133.5 153.  166.  202. ]
oldpeak  categories with bin edges:  [0.  0.8 1.6 6.2]


In [37]:
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,3,1,3,3,1,1,0,1,0,2,0,0,1,1
1,0,1,2,1,2,0,1,3,0,2,0,0,2,1
2,0,0,1,1,0,0,0,3,0,1,2,0,2,1
3,2,1,1,0,1,0,1,3,0,0,2,0,2,1
4,2,0,0,0,3,0,1,2,1,0,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,2,0,0,2,2,0,1,0,1,0,1,0,3,0
299,0,1,3,0,2,0,1,0,0,1,1,0,3,0
300,3,1,0,3,0,1,1,1,0,2,1,2,3,0
301,2,1,0,1,0,0,1,0,1,1,1,1,3,0


In [38]:
cutoff = 0.80
num_train = round(cutoff * len(df))
num_test = len(df) - num_train

print("Training data points: ", num_train)
print("Testing data points: ", num_test)

df = df.sample(frac=1)

trainY = np.array(df.iloc[: num_train]["target"])
testY = np.array(df.iloc[num_train:]["target"])

df = df.drop(["target"], axis = 1)

trainX = np.array(df.iloc[: num_train, :])
testX = np.array(df.iloc[num_train: , :])

Training data points:  242
Testing data points:  61


In [50]:
clf = RandomForestClassifier(max_depth=4, random_state=0)
clf.fit(trainX, trainY)
print("Training Accuracy: ", clf.score(trainX, trainY))
print("Testing Accuracy: ", clf.score(testX, testY))
print("F1 Score: ", f1_score(testY, clf.predict(testX)))

Training Accuracy:  0.8884297520661157
Testing Accuracy:  0.819672131147541
F1 Score:  0.8307692307692307


In [54]:
# neural network

model = keras.Sequential([
    tf.keras.layers.Dense(32, input_dim = trainX.shape[1], activation = 'relu', kernel_regularizer=keras.regularizers.l2(0.1)),
    tf.keras.layers.Dense(16, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

epoch = 1000
model.summary()
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[f1_metric])
acc = model.fit(trainX, trainY, validation_split = 0.1, epochs = epoch, batch_size = trainX.shape[0])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 32)                448       
_________________________________________________________________
dense_4 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 17        
Total params: 993
Trainable params: 993
Non-trainable params: 0
_________________________________________________________________
Epoch 1/1000
1/1 [==============================] - 1s 521ms/step - loss: 2.4123 - f1_metric: 0.7073 - val_loss: 2.3890 - val_f1_metric: 0.7222
Epoch 2/1000
1/1 [==============================] - 0s 33ms/step - loss: 2.3933 - f1_metric: 0.7095 - val_loss: 2.3700 - val_f1_metric: 0.7222
Epoch 3/1000
1/1 [==============================] - 0s 39ms/step - loss: 2.3747 - f1_metric: 0.7037

Epoch 53/1000
1/1 [==============================] - 0s 35ms/step - loss: 1.6393 - f1_metric: 0.8561 - val_loss: 1.6460 - val_f1_metric: 0.8276
Epoch 54/1000
1/1 [==============================] - 0s 39ms/step - loss: 1.6270 - f1_metric: 0.8561 - val_loss: 1.6342 - val_f1_metric: 0.8276
Epoch 55/1000
1/1 [==============================] - 0s 37ms/step - loss: 1.6148 - f1_metric: 0.8561 - val_loss: 1.6224 - val_f1_metric: 0.8276
Epoch 56/1000
1/1 [==============================] - 0s 37ms/step - loss: 1.6027 - f1_metric: 0.8593 - val_loss: 1.6106 - val_f1_metric: 0.8276
Epoch 57/1000
1/1 [==============================] - 0s 36ms/step - loss: 1.5907 - f1_metric: 0.8593 - val_loss: 1.5988 - val_f1_metric: 0.8276
Epoch 58/1000
1/1 [==============================] - 0s 34ms/step - loss: 1.5787 - f1_metric: 0.8550 - val_loss: 1.5872 - val_f1_metric: 0.7857
Epoch 59/1000
1/1 [==============================] - 0s 41ms/step - loss: 1.5667 - f1_metric: 0.8582 - val_loss: 1.5755 - val_f1_metric:

Epoch 110/1000
1/1 [==============================] - 0s 43ms/step - loss: 1.0830 - f1_metric: 0.8740 - val_loss: 1.0979 - val_f1_metric: 0.8571
Epoch 111/1000
1/1 [==============================] - 0s 36ms/step - loss: 1.0756 - f1_metric: 0.8740 - val_loss: 1.0907 - val_f1_metric: 0.8571
Epoch 112/1000
1/1 [==============================] - 0s 35ms/step - loss: 1.0684 - f1_metric: 0.8740 - val_loss: 1.0836 - val_f1_metric: 0.8571
Epoch 113/1000
1/1 [==============================] - 0s 34ms/step - loss: 1.0612 - f1_metric: 0.8740 - val_loss: 1.0766 - val_f1_metric: 0.8966
Epoch 114/1000
1/1 [==============================] - 0s 45ms/step - loss: 1.0541 - f1_metric: 0.8775 - val_loss: 1.0697 - val_f1_metric: 0.8966
Epoch 115/1000
1/1 [==============================] - 0s 46ms/step - loss: 1.0470 - f1_metric: 0.8775 - val_loss: 1.0628 - val_f1_metric: 0.8966
Epoch 116/1000
1/1 [==============================] - 0s 44ms/step - loss: 1.0401 - f1_metric: 0.8775 - val_loss: 1.0561 - val_f1_

1/1 [==============================] - 0s 35ms/step - loss: 0.5781 - f1_metric: 0.8845 - val_loss: 0.6096 - val_f1_metric: 0.9333
Epoch 223/1000
1/1 [==============================] - 0s 51ms/step - loss: 0.5755 - f1_metric: 0.8845 - val_loss: 0.6068 - val_f1_metric: 0.9333
Epoch 224/1000
1/1 [==============================] - 0s 40ms/step - loss: 0.5729 - f1_metric: 0.8845 - val_loss: 0.6040 - val_f1_metric: 0.9333
Epoch 225/1000
1/1 [==============================] - 0s 42ms/step - loss: 0.5704 - f1_metric: 0.8845 - val_loss: 0.6014 - val_f1_metric: 0.9333
Epoch 226/1000
1/1 [==============================] - 0s 36ms/step - loss: 0.5679 - f1_metric: 0.8845 - val_loss: 0.5988 - val_f1_metric: 0.9333
Epoch 227/1000
1/1 [==============================] - 0s 34ms/step - loss: 0.5654 - f1_metric: 0.8845 - val_loss: 0.5962 - val_f1_metric: 0.9333
Epoch 228/1000
1/1 [==============================] - 0s 41ms/step - loss: 0.5629 - f1_metric: 0.8845 - val_loss: 0.5936 - val_f1_metric: 0.9333


1/1 [==============================] - 0s 39ms/step - loss: 0.3785 - f1_metric: 0.9040 - val_loss: 0.4621 - val_f1_metric: 0.9032
Epoch 335/1000
1/1 [==============================] - 0s 41ms/step - loss: 0.3773 - f1_metric: 0.9040 - val_loss: 0.4612 - val_f1_metric: 0.9032
Epoch 336/1000
1/1 [==============================] - 0s 43ms/step - loss: 0.3761 - f1_metric: 0.9040 - val_loss: 0.4606 - val_f1_metric: 0.9032
Epoch 337/1000
1/1 [==============================] - 0s 36ms/step - loss: 0.3750 - f1_metric: 0.9040 - val_loss: 0.4600 - val_f1_metric: 0.9032
Epoch 338/1000
1/1 [==============================] - 0s 38ms/step - loss: 0.3737 - f1_metric: 0.9040 - val_loss: 0.4594 - val_f1_metric: 0.9032
Epoch 339/1000
1/1 [==============================] - 0s 36ms/step - loss: 0.3725 - f1_metric: 0.9040 - val_loss: 0.4589 - val_f1_metric: 0.9032
Epoch 340/1000
1/1 [==============================] - 0s 38ms/step - loss: 0.3712 - f1_metric: 0.9040 - val_loss: 0.4588 - val_f1_metric: 0.9032


1/1 [==============================] - 0s 34ms/step - loss: 0.2837 - f1_metric: 0.9120 - val_loss: 0.4601 - val_f1_metric: 0.8276
Epoch 447/1000
1/1 [==============================] - 0s 39ms/step - loss: 0.2830 - f1_metric: 0.9120 - val_loss: 0.4602 - val_f1_metric: 0.8276
Epoch 448/1000
1/1 [==============================] - 0s 41ms/step - loss: 0.2824 - f1_metric: 0.9120 - val_loss: 0.4604 - val_f1_metric: 0.8276
Epoch 449/1000
1/1 [==============================] - 0s 34ms/step - loss: 0.2819 - f1_metric: 0.9163 - val_loss: 0.4601 - val_f1_metric: 0.8276
Epoch 450/1000
1/1 [==============================] - 0s 34ms/step - loss: 0.2813 - f1_metric: 0.9163 - val_loss: 0.4598 - val_f1_metric: 0.8276
Epoch 451/1000
1/1 [==============================] - 0s 33ms/step - loss: 0.2807 - f1_metric: 0.9163 - val_loss: 0.4605 - val_f1_metric: 0.8276
Epoch 452/1000
1/1 [==============================] - 0s 33ms/step - loss: 0.2802 - f1_metric: 0.9163 - val_loss: 0.4613 - val_f1_metric: 0.8276


1/1 [==============================] - 0s 34ms/step - loss: 0.2295 - f1_metric: 0.9431 - val_loss: 0.4664 - val_f1_metric: 0.8276
Epoch 559/1000
1/1 [==============================] - 0s 37ms/step - loss: 0.2290 - f1_metric: 0.9474 - val_loss: 0.4662 - val_f1_metric: 0.8276
Epoch 560/1000
1/1 [==============================] - 0s 34ms/step - loss: 0.2285 - f1_metric: 0.9474 - val_loss: 0.4663 - val_f1_metric: 0.8276
Epoch 561/1000
1/1 [==============================] - 0s 32ms/step - loss: 0.2281 - f1_metric: 0.9474 - val_loss: 0.4677 - val_f1_metric: 0.8276
Epoch 562/1000
1/1 [==============================] - 0s 33ms/step - loss: 0.2277 - f1_metric: 0.9431 - val_loss: 0.4670 - val_f1_metric: 0.8276
Epoch 563/1000
1/1 [==============================] - 0s 35ms/step - loss: 0.2274 - f1_metric: 0.9431 - val_loss: 0.4686 - val_f1_metric: 0.8276
Epoch 564/1000
1/1 [==============================] - 0s 36ms/step - loss: 0.2270 - f1_metric: 0.9474 - val_loss: 0.4690 - val_f1_metric: 0.8276


1/1 [==============================] - 0s 36ms/step - loss: 0.1900 - f1_metric: 0.9714 - val_loss: 0.5154 - val_f1_metric: 0.8571
Epoch 671/1000
1/1 [==============================] - 0s 40ms/step - loss: 0.1898 - f1_metric: 0.9672 - val_loss: 0.5159 - val_f1_metric: 0.8276
Epoch 672/1000
1/1 [==============================] - 0s 35ms/step - loss: 0.1894 - f1_metric: 0.9675 - val_loss: 0.5097 - val_f1_metric: 0.8571
Epoch 673/1000
1/1 [==============================] - 0s 33ms/step - loss: 0.1892 - f1_metric: 0.9672 - val_loss: 0.5087 - val_f1_metric: 0.8276
Epoch 674/1000
1/1 [==============================] - 0s 43ms/step - loss: 0.1891 - f1_metric: 0.9636 - val_loss: 0.5107 - val_f1_metric: 0.8571
Epoch 675/1000
1/1 [==============================] - 0s 33ms/step - loss: 0.1886 - f1_metric: 0.9714 - val_loss: 0.5181 - val_f1_metric: 0.8276
Epoch 676/1000
1/1 [==============================] - 0s 35ms/step - loss: 0.1882 - f1_metric: 0.9714 - val_loss: 0.5233 - val_f1_metric: 0.8276


1/1 [==============================] - 0s 36ms/step - loss: 0.1610 - f1_metric: 0.9794 - val_loss: 0.5535 - val_f1_metric: 0.8571
Epoch 783/1000
1/1 [==============================] - 0s 36ms/step - loss: 0.1609 - f1_metric: 0.9794 - val_loss: 0.5520 - val_f1_metric: 0.8571
Epoch 784/1000
1/1 [==============================] - 0s 35ms/step - loss: 0.1609 - f1_metric: 0.9794 - val_loss: 0.5518 - val_f1_metric: 0.8571
Epoch 785/1000
1/1 [==============================] - 0s 35ms/step - loss: 0.1604 - f1_metric: 0.9794 - val_loss: 0.5572 - val_f1_metric: 0.8571
Epoch 786/1000
1/1 [==============================] - 0s 36ms/step - loss: 0.1602 - f1_metric: 0.9794 - val_loss: 0.5602 - val_f1_metric: 0.8571
Epoch 787/1000
1/1 [==============================] - 0s 35ms/step - loss: 0.1600 - f1_metric: 0.9794 - val_loss: 0.5603 - val_f1_metric: 0.8571
Epoch 788/1000
1/1 [==============================] - 0s 32ms/step - loss: 0.1598 - f1_metric: 0.9794 - val_loss: 0.5607 - val_f1_metric: 0.8571


1/1 [==============================] - 0s 42ms/step - loss: 0.1398 - f1_metric: 0.9917 - val_loss: 0.5951 - val_f1_metric: 0.8571
Epoch 895/1000
1/1 [==============================] - 0s 37ms/step - loss: 0.1397 - f1_metric: 0.9836 - val_loss: 0.5939 - val_f1_metric: 0.8571
Epoch 896/1000
1/1 [==============================] - 0s 42ms/step - loss: 0.1396 - f1_metric: 0.9917 - val_loss: 0.5946 - val_f1_metric: 0.8571
Epoch 897/1000
1/1 [==============================] - 0s 36ms/step - loss: 0.1391 - f1_metric: 0.9917 - val_loss: 0.5979 - val_f1_metric: 0.8571
Epoch 898/1000
1/1 [==============================] - 0s 36ms/step - loss: 0.1391 - f1_metric: 0.9917 - val_loss: 0.6022 - val_f1_metric: 0.8571
Epoch 899/1000
1/1 [==============================] - 0s 38ms/step - loss: 0.1391 - f1_metric: 0.9917 - val_loss: 0.6017 - val_f1_metric: 0.8571
Epoch 900/1000
1/1 [==============================] - 0s 35ms/step - loss: 0.1393 - f1_metric: 0.9836 - val_loss: 0.6007 - val_f1_metric: 0.8571


In [55]:
print("F1 for a two hidden-layer Neural Netowrk: ", model.evaluate(trainX, trainY)[1])

8/8 [==============================] - 0s 1ms/step - loss: 0.1772 - f1_metric: 0.9729
F1 for a two hidden-layer Neural Netowrk:  0.9729201793670654
